In [1]:
# https://www.kaggle.com/code/arjunjoshua/predicting-fraud-in-financial-payment-services/notebook


In [2]:
import os
os.chdir('../')

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
import warnings
from sklearn.base import BaseEstimator, TransformerMixin
import src.transformers as trans
from collections import defaultdict

warnings.filterwarnings("ignore", category=DeprecationWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [377]:
DEBUG = int(5e5)
TARGET = "isFraud"


df_train = pd.read_csv('./data/fraud.csv', nrows=DEBUG)
df_train = df_train.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})                        


df_train_model = trans.SelectToModel().transform(df_train).sample(frac=1, random_state=42)

y_train = df_train_model.pop(TARGET)
X_train = df_train_model

if DEBUG:
    df_test = pd.read_csv('./data/fraud.csv', header=None, skiprows=DEBUG, nrows=2e5)
    df_test.columns = df_train.columns
    df_test = df_test.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                            'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})       

    df_test_model = trans.SelectToModel().transform(df_test)

    y_test = df_test_model.pop(TARGET)
    X_test = df_test_model                 







In [378]:
X_train.shape,y_train.shape , X_test.shape,y_test.shape

((223046, 10), (223046,), (84678, 10), (84678,))

# EDA

In [360]:
# df_transfer = df.query("isFraud == 1 & type == 'TRANSFER'")
# df_cashout = df.query("isFraud == 1 & type == 'CASH_OUT'")

# len(df_transfer) , len(df_cashout)


In [361]:
# randomState = 5
# np.random.seed(randomState)
# X_sample = X.sample(frac=.01)

# Y = X['isFraud']
# del X['isFraud']

In [362]:
# PARAM = 200
# X.loc[(X['isFraud']==0) & (X['step']<PARAM),'step'].hist(bins=PARAM, density=True, alpha=.5)
# X.loc[(X['isFraud']==1) & (X['step']<PARAM),'step'].hist(bins=PARAM, density=True, alpha=.5)

In [363]:

# sns.catplot(data=X_sample, x="step", y='isFraud', orient='h', hue="type",  aspect=2, alpha=.3 )

In [364]:
# sns.catplot(data=X_sample, x="amount", y='isFraud', orient='h', hue="type",  aspect=2, alpha=.3 )

In [365]:
# sns.catplot(data=X_sample, y="errorBalanceDest", x='isFraud', orient='v', hue="type",  aspect=2, alpha=.3, height=5 )

# Model

In [562]:
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from lightgbm import LGBMClassifier

DROP_COLUMNS = ['nameOrig', 'nameDest', 'isFlaggedFraud']



transformation_pipeline = Pipeline([
    ('drop_columns', trans.DropColumns(DROP_COLUMNS)),
    ('label_encode', trans.CustomLabelEncoder(['type'])),
    ('amount_vs_old_new_balance_dest', trans.AmountVsOldAndNewBalanceDest()),
    ('amount_vs_old_new_balance_orig', trans.AmountVsOldAndNewBalanceOrig())
])

weights = sum(df_train[TARGET] == 0) / sum(df_train[TARGET] == 1)
# weights = np.mean(df_train[TARGET])


# weights = [1, 1/10, 1/25, 1/50, 1/75, 1/99, 1/100, 1/1000]
# param_grid = dict(scale_pos_weight=weights)


model_pipeline = Pipeline([
    # ('xgbclassifier', XGBClassifier(max_depth = 1, scale_pos_weight = weights, n_jobs = 4)) ,
    ('lgbmclassifier', LGBMClassifier(max_debth= 3, metric='AUC', scale_pos_weight = 1/weights, n_jobs=-1))
    ])


pipe = Pipeline([
    ('transformation', transformation_pipeline),
    ('model', model_pipeline)
])

# grid = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

TimeSeriesSplit(pipe, X_train, y_train)


#https://discuss.xgboost.ai/t/how-does-scale-pos-weight-affect-probabilities/1790

TypeError: __init__() takes from 1 to 2 positional arguments but 4 were given

In [563]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=48,
    max_train_size=10000,
    test_size=1000,
)

In [564]:
ts_cv

TimeSeriesSplit(gap=48, max_train_size=10000, n_splits=5, test_size=1000)

In [521]:
pipe.fit(X_train, y_train)
probabilities = pipe.predict_proba(X_test)
print('AUPRC = {}'.format(average_precision_score(y_test, probabilities[:, 1]>.5)))

[LightGBM] [Warning] Unknown parameter: max_debth
AUPRC = 0.9349243220096181


In [554]:
# from tqdm import tqdm
# for i in tqdm(range(X_test.shape[0])[7000:]):
#     # print(i)
#     probabilities = pipe.predict_proba(X_test.iloc[[i]])
#     if y_test.iloc[i] == 1:
#         print('Fraud!')
#         print(probabilities[0][1]>.5)
#     else:
#         if probabilities[0][1]>.5:
#             print('False Positive!!')